In [125]:
from qiskit.algorithms import VQE
from qiskit_nature.algorithms import (GroundStateEigensolver,
                                      NumPyMinimumEigensolverFactory)
import matplotlib.pyplot as plt
import numpy as np
from qiskit_nature.circuit.library import UCCSD, HartreeFock
from qiskit.circuit.library import EfficientSU2
from qiskit.algorithms.optimizers import COBYLA, SPSA, SLSQP
from qiskit import IBMQ, BasicAer, Aer
from qiskit.utils import QuantumInstance
from qiskit.utils.mitigation import CompleteMeasFitter
from qiskit.providers.aer.noise import NoiseModel

def exact_solver(problem, converter):
    solver = NumPyMinimumEigensolverFactory()
    calc = GroundStateEigensolver(converter, solver)
    result = calc.solve(problem)
    return result

def calc_energy(op,num_part,num_orb,problem,converter):
    
    backend = BasicAer.get_backend("statevector_simulator")

    #no clue why this is needed. Without it the initial state has different # of qubits than the number of qubits in qubit operator
    # and we get an error. 
    result = exact_solver(problem,converter)

    optimizer = SLSQP(maxiter=5)

        #result = exact_solver(problem,converter)
        #exact_energies.append(result.total_energies[0].real)
    
    init_state = HartreeFock(num_orb, num_part, converter)
   
    var_form = UCCSD(converter,
                        num_part,
                        num_orb,
                        initial_state=init_state)
    vqe = VQE(var_form, optimizer, quantum_instance=backend)
    
    vqe_calc = vqe.compute_minimum_eigenvalue(op)
    vqe_result = problem.interpret(vqe_calc).total_energies[0].real
    return vqe_result  

In [126]:
from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureMoleculeDriver, ElectronicStructureDriverType)
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.transformers.second_quantization.electronic import FreezeCoreTransformer
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import ParityMapper
from qiskit.opflow import TwoQubitReduction



def get_qubit_op(coordinates):

    
    # Define Molecule
    molecule = Molecule(
        # Coordinates in Angstrom
        geometry=[
            ["H", [coordinates[0], 0.0, 0.0] ],
            ["H", [coordinates[1], 0.0, 0.0] ]
        ],
        multiplicity=1,  # = 2*spin + 1
        charge=0,
    )

    driver = ElectronicStructureMoleculeDriver(
        molecule=molecule,
        basis="sto3g",
        driver_type=ElectronicStructureDriverType.PYSCF)

    # Get properties
    properties = driver.run()
    num_particles = (properties
                        .get_property("ParticleNumber")
                        .num_particles)
    num_spin_orbitals = int(properties
                            .get_property("ParticleNumber")
                            .num_spin_orbitals)
    
    # Define Problem, Use freeze core approximation, remove orbitals.
    problem = ElectronicStructureProblem(
        driver,
        [FreezeCoreTransformer(freeze_core=True)])

    second_q_ops = problem.second_q_ops()  # Get 2nd Quant OP
    num_spin_orbitals = problem.num_spin_orbitals
    num_particles = problem.num_particles

    mapper = ParityMapper()  # Set Mapper
    hamiltonian = second_q_ops[0]  # Set Hamiltonian
    # Do two qubit reduction
    converter = QubitConverter(mapper,two_qubit_reduction=True)
    reducer = TwoQubitReduction(num_particles)
    qubit_op = converter.convert(hamiltonian)
    qubit_op = reducer.convert(qubit_op)

    return qubit_op, num_particles, num_spin_orbitals, problem, converter

def calc_ground_state(op,num_part,num_orb,problem,converter):

    backend = BasicAer.get_backend("statevector_simulator")
 
    result = exact_solver(problem,converter)

    optimizer = SLSQP(maxiter=5)

    init_state = HartreeFock(num_orb, num_part, converter)
     
    var_form = UCCSD(converter,
                        num_part,
                        num_orb,
                        initial_state=init_state)

    vqe = VQE(var_form, optimizer, quantum_instance=backend) 
    vqe_result = vqe.compute_minimum_eigenvalue(op)
    min_eng = vqe_result.eigenvalue
    #vqe_ground = vqe_result.eigenstate perhaps more accurate? Downside: don't get circuit 
    final_params = vqe_result.optimal_parameters 

    vqe_ground = vqe.ansatz.bind_parameters(final_params)  
    
    return vqe_ground, min_eng

In [127]:
'''
### Testing the calc_gound_state function

from qiskit.providers.aer import QasmSimulator
from qiskit import QuantumCircuit, transpile
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit.opflow import PauliExpectation, CircuitSampler, StateFn, CircuitStateFn
from qiskit.quantum_info.operators import Operator


(qubit_op, num_particles, num_spin_orbitals, problem, converter) = get_qubit_op([0,1])

psi_0,min_eng = calc_ground_state(qubit_op,num_particles, num_spin_orbitals, problem, converter)

backend = Aer.get_backend('qasm_simulator') 
q_instance = QuantumInstance(backend, shots=8024)

psi_0 = CircuitStateFn(psi_0)
measurable_expression = StateFn(qubit_op, is_measurement=True).compose(psi_0) 
expectation = PauliExpectation().convert(measurable_expression)  

# get state sampler (you can also pass the backend directly)
sampler = CircuitSampler(q_instance).convert(expectation) 
E0 = sampler.eval().real
'''

"\n### Testing the calc_gound_state function\n\nfrom qiskit.providers.aer import QasmSimulator\nfrom qiskit import QuantumCircuit, transpile\nfrom qiskit import Aer\nfrom qiskit.utils import QuantumInstance\nfrom qiskit.opflow import PauliExpectation, CircuitSampler, StateFn, CircuitStateFn\nfrom qiskit.quantum_info.operators import Operator\n\n\n(qubit_op, num_particles, num_spin_orbitals, problem, converter) = get_qubit_op([0,1])\n\npsi_0,min_eng = calc_ground_state(qubit_op,num_particles, num_spin_orbitals, problem, converter)\n\nbackend = Aer.get_backend('qasm_simulator') \nq_instance = QuantumInstance(backend, shots=8024)\n\npsi_0 = CircuitStateFn(psi_0)\nmeasurable_expression = StateFn(qubit_op, is_measurement=True).compose(psi_0) \nexpectation = PauliExpectation().convert(measurable_expression)  \n\n# get state sampler (you can also pass the backend directly)\nsampler = CircuitSampler(q_instance).convert(expectation) \nE0 = sampler.eval().real\n"

In [128]:
eV = 1.602e-19
angst = 1e-10
dR = 0.02*angst

def calc_forces(coordinates):
    
    #get qubit operator corresponding to H(R)
    (qubit_op, _, _, _, _) = get_qubit_op([coordinates[0],coordinates[1]])
    
    #get qubit operators coresponding to H+ (hamiltonian for atomic distances R + dR)
    (qubit_op_plus_0, _, _, _, _) = get_qubit_op([coordinates[0] + dR,coordinates[1]])
    
    #get qubit operators coresponding to H- (hamiltonian for atomic distances R - dR)
    (qubit_op_minus_0, num_particles, num_spin_orbitals, problem, converter) = get_qubit_op([coordinates[0] - dR,coordinates[1]])
 
    #get the ground state of H(R)
    psi_0,_ = calc_ground_state(qubit_op,num_particles, num_spin_orbitals, problem, converter)
    
    #define desired observable (H_+ - H_-)/(2dR) = force  
    Obs0 = (qubit_op_plus_0-qubit_op_minus_0)*(1/(2*dR)*eV)
       
    #get the expectation value <psi_0|O|psi_0>¨
    
    backend = Aer.get_backend('qasm_simulator') 
    q_instance = QuantumInstance(backend, shots=8024)

    psi_0 = CircuitStateFn(psi_0)
    measurable_expression = StateFn(Obs0, is_measurement=True).compose(psi_0) 
    expectation = PauliExpectation().convert(measurable_expression)  
    sampler = CircuitSampler(q_instance).convert(expectation) 
    f0 = sampler.eval().real
    
    ### repeat same for other atom ###
    (qubit_op_plus_1, _, _, _, _) = get_qubit_op([coordinates[0],coordinates[1]+dR])
    
    (qubit_op_minus_1, num_particles, num_spin_orbitals, problem, converter) = get_qubit_op([coordinates[0],coordinates[1]-dR])

    #define desired observable (H_+ - H_-)/(2dR) = force  
    Obs1 = (qubit_op_plus_1-qubit_op_minus_1)*(1/(2*dR)*eV)
    
    backend = Aer.get_backend('qasm_simulator') 
    q_instance = QuantumInstance(backend, shots=1)
    measurable_expression = StateFn(Obs1, is_measurement=True).compose(psi_0) 
    expectation = PauliExpectation().convert(measurable_expression)  
    sampler = CircuitSampler(q_instance).convert(expectation) 
    f1 = sampler.eval().real
    
    return f0,f1

In [ ]:
#implementation of the simple Verlet integrator
femto = 1e-15
angst = 1e-10


#mass of particle1 (H)
mass_0 = 1.67e-27
#mass of particle2 (H)
mass_1 = 1.67e-27
#time step of integrator
dt = 0.2*femto
#initial velocity of atoms
v_init_0 = 0
v_init_1 = 0

#initial positions of atoms
init_pos = [0,1]

#integrator timesteps
times = np.arange(0*femto, 4.0*femto, dt)
#coordinate array
coords = [init_pos]

for time in times:
    r = coords[-1]
    (f0,f1) = calc_forces(coords[-1])
    if time == 0:
        r0_next = r[0]*angst + v_init_0*dt + 0.5*f0/mass_0*(dt*dt)
        r1_next = r[1]*angst + v_init_1*dt + 0.5*f1/mass_1*(dt*dt)
        coords.append([r0_next/angst,r1_next/angst])
    else:
        r_prev = coords[-2]
        r0_next = 2*r[0]*angst - r_prev[0]*angst + f0/mass_0*(dt*dt)
        r1_next = 2*r[1]*angst - r_prev[1]*angst + f1/mass_1*(dt*dt)
        coords.append([r0_next/angst,r1_next/angst])
    print(coords)

[[0, 1], [0.0, 1.0]]
[[0, 1], [0.0, 1.0], [0.0, 1.0]]
[[0, 1], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0]]
[[0, 1], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0]]
[[0, 1], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0]]
[[0, 1], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0]]
[[0, 1], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0]]
[[0, 1], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0]]
